<a href="https://colab.research.google.com/github/OleksiiLatypov/DRU_Data_Science_Fundamentals/blob/main/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import numpy as np
import pandas as pd
import re as re


train = pd.read_csv('train.csv', header = 0, dtype={'Age': np.float64})
val  = pd.read_csv('val.csv' , header = 0, dtype={'Age': np.float64})
full_data = [train, val]

# url = 'https://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv'
# train = pd.read_csv(url)

train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean()

,Pclass,Survived
0,1,0.629630
1,2,0.472826
2,3,0.242363


In [6]:
train[["Sex", "Survived"]].groupby(['Sex'], as_index = False).mean()

,Sex,Survived
0,female,0.742038
1,male,0.188908


In [7]:
train['FamilySize'] = train['SibSp'] + train['Parch'] + 1
train.head()

train[['FamilySize', 'Survived']].groupby(['FamilySize']).mean()

,Survived
FamilySize,
1,0.303538
2,0.552795
3,0.578431
4,0.724138
5,0.200000
6,0.136364
7,0.333333
8,0.000000
11,0.000000


In [8]:
train['IsAlone'] = 0
train.loc[train['FamilySize'] == 1, 'IsAlone'] = 1
train[['IsAlone', 'Survived']].groupby(['IsAlone'], as_index=False).mean()

,IsAlone,Survived
0,0,0.505650
1,1,0.303538


In [9]:
train['Embarked'] = train['Embarked'].fillna(train['Embarked'].mode()[0])
train.head()
train[['Embarked', 'Survived']].groupby(['Embarked']).mean()

,Survived
Embarked,
C,0.553571
Q,0.389610
S,0.339009


**Fare**: Fare also has some missing values and we will replace them with the median. Then, we categorize it into 4 ranges.

In [10]:
train['Fare'] = train['Fare'].fillna(train['Fare'].median())
train['CategoricalFare'] = pd.qcut(train['Fare'], 4)
train[['CategoricalFare', 'Survived']].groupby(['CategoricalFare'], as_index=False).mean()

,CategoricalFare,Survived
0,"(-0.001, 7.91]",0.197309
1,"(7.91, 14.454]",0.303571
2,"(14.454, 31.0]",0.454955
3,"(31.0, 512.329]",0.581081


As we can see, passengers with the cheapest tickets had almost no chance to survive and the probability to survive grew with the price of the ticket.

**Age**: we have plenty of missing values in this feature. Let's generate random numbers between (mean - std) and (mean + std) to fill Nans, then we will categorize age into 5 range:

In [11]:
age_avg = train['Age'].mean()
age_std = train['Age'].std()
age_null_count = train['Age'].isnull().sum()
# print(age_avg)
# print(age_std)


age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size=age_null_count)
train['Age'][np.isnan(train['Age'])] = age_null_random_list
train['Age'] = train['Age'].astype('int')


train['CategoricalAge'] = pd.cut(train['Age'], 5)
train[['CategoricalAge', 'Survived']].groupby(['CategoricalAge'], as_index=False).mean()


<ipython-input-11-4220f7379c11>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['Age'][np.isnan(train['Age'])] = age_null_random_list


,CategoricalAge,Survived
0,"(-0.08, 16.0]",0.518519
1,"(16.0, 32.0]",0.349451
2,"(32.0, 48.0]",0.387097
3,"(48.0, 64.0]",0.434783
4,"(64.0, 80.0]",0.090909


We can see that children had the highest chance to survive.

**Name**: inside this feature we can find passengers' titles.

In [12]:
def get_title(name:str) -> str:
  title_search = re.search(' ([A-Za-z]+)\.', name)
  # If the title exists, extract and return it.
  if title_search:
    return title_search.group(1)
  return ""

train['Title'] = train['Name'].apply(get_title)
pd.crosstab(train['Title'], train['Sex'])

Sex,female,male
Title,,
Capt,0,1
Col,0,2
Countess,1,0
Don,0,1
Dr,1,6
Jonkheer,0,1
Lady,1,0
Major,0,2
Master,0,40


In [13]:
train['Title'] = train['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
    'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')


train['Title'] = train['Title'].replace('Mlle', 'Miss')
train['Title'] = train['Title'].replace('Ms', 'Miss')
train['Title'] = train['Title'].replace('Mme', 'Mrs')

train[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

,Title,Survived
0,Master,0.575000
1,Miss,0.702703
2,Mr,0.156673
3,Mrs,0.793651
4,Rare,0.347826


As we can judge, young ladies had more chances to survive than other people.

**Data Preprocessing**

Let's process the data taking into the account the insights we got from Feature Engineering.

Now we will define processing operations and their descriptions regarding mentioned possible operations, which we will use in specifications.json:

**1. columns combinations:**

In [14]:
train['FamilySize'] = train['SibSp'] + train['Parch'] + 1

**2. replace value**

In [15]:
train['IsAlone'] = 0
train.loc[train['FamilySize'] == 1, 'IsAlone'] = 1

**3. fill Nan with mode**

In [16]:
train['Embarked'] = train['Embarked'].fillna(train['Embarked'].mode()[0])

**4. fill Nan with median**

In [17]:
train['Fare'] = train['Fare'].fillna(train['Fare'].median())

**5. binning with qcut**

In [18]:
train['Fare'] = pd.qcut(train['Fare'], 4)

**6. fill Nan with values from random distribution**

In [19]:
age_avg = train['Age'].mean()
age_std = train['Age'].std()
age_null_count = train['Age'].isnull().sum()
rng = np.random.RandomState(42)
age_null_random_list = rng.uniform(age_avg - age_std, age_avg + age_std, size=age_null_count)
train['Age'][np.isnan(train['Age'])] = age_null_random_list

<ipython-input-19-d16a66177ae5>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['Age'][np.isnan(train['Age'])] = age_null_random_list


**7. binning with cut**

In [20]:
train['Age'] = pd.cut(train['Age'], 5)

**8. apply regex using previously mentioned get_title function:**

In [21]:
train['Title'] = train['Name'].apply(get_title)

**9. replace value**

In [22]:
train['Title'] = train['Title'].replace(['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr',
                                        'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'],
                                        'Rare')

**10. replace value**

In [23]:
train['Title'] = train['Title'].replace(['Mlle', 'Ms'], 'Miss')

**11. replace value**

In [24]:
train['Title'] = train['Title'].replace('Mme', 'Mrs')

**12. fill Nans with 0**

In [25]:
train['Title'] = train['Title'].fillna(0)

**13. drop columns**

In [26]:
drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin',
                 'SibSp', 'Parch', 'FamilySize']
train = train.drop(drop_elements, axis=1)

In [27]:
train

,Survived,Pclass,Sex,Age,Fare,Embarked,IsAlone,CategoricalFare,CategoricalAge,Title
0,0,3,male,"(16.0, 32.0]","(-0.001, 7.91]",S,0,"(-0.001, 7.91]","(16.0, 32.0]",Mr
1,1,1,female,"(32.0, 48.0]","(31.0, 512.329]",C,0,"(31.0, 512.329]","(32.0, 48.0]",Mrs
2,1,3,female,"(16.0, 32.0]","(7.91, 14.454]",S,1,"(7.91, 14.454]","(16.0, 32.0]",Miss
3,1,1,female,"(32.0, 48.0]","(31.0, 512.329]",S,0,"(31.0, 512.329]","(32.0, 48.0]",Mrs
4,0,3,male,"(32.0, 48.0]","(7.91, 14.454]",S,1,"(7.91, 14.454]","(32.0, 48.0]",Mr
...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,"(16.0, 32.0]","(7.91, 14.454]",S,1,"(7.91, 14.454]","(16.0, 32.0]",Rare
887,1,1,female,"(16.0, 32.0]","(14.454, 31.0]",S,1,"(14.454, 31.0]","(16.0, 32.0]",Miss
888,0,3,female,"(32.0, 48.0]","(14.454, 31.0]",S,0,"(14.454, 31.0]","(32.0, 48.0]",Miss
889,1,1,male,"(16.0, 32.0]","(14.454, 31.0]",C,1,"(14.454, 31.0]","(16.0, 32.0]",Mr


In [28]:
from sklearn.preprocessing import LabelEncoder

# encode labels
le = LabelEncoder()

le.fit(train['Sex'])
train['Sex'] = le.transform(train['Sex'])

le.fit(train['Title'])
train['Title'] = le.transform(train['Title'])

le.fit(train['Embarked'].values)
train['Embarked'] = le.transform(train['Embarked'].values)

le.fit(train['Fare'])
train['Fare'] = le.transform(train['Fare'])

le.fit(train['Age'])
train['Age'] = le.transform(train['Age'])

In [30]:
train.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,IsAlone,CategoricalFare,CategoricalAge,Title
0,0,3,1,1,0,2,0,"(-0.001, 7.91]","(16.0, 32.0]",2
1,1,1,0,2,3,0,0,"(31.0, 512.329]","(32.0, 48.0]",3
2,1,3,0,1,1,2,1,"(7.91, 14.454]","(16.0, 32.0]",1
3,1,1,0,2,3,2,0,"(31.0, 512.329]","(32.0, 48.0]",3
4,0,3,1,2,1,2,1,"(7.91, 14.454]","(32.0, 48.0]",2


Great! Now that we have defined all preprocessing operations, let's gather them under the Dataloader class:

utils/dataloader.py

In [33]:
import pandas as pd
import numpy as np
import re

from sklearn.preprocessing import LabelEncoder


class DataLoader(object):
    def fit(self, dataset):
        self.dataset = dataset.copy()

    # apply regex
    def get_title(self, name):
        pattern = ' ([A-Za-z]+)\.'
        title_search = re.search(pattern, name)
        # If the title exists, extract and return it.
        if title_search:
            return title_search.group(1)
        return ""

    def load_data(self):
        # columns combination
        self.dataset['FamilySize'] = self.dataset['SibSp'] + self.dataset['Parch'] + 1

        # replace value
        self.dataset['IsAlone'] = 0
        self.dataset.loc[self.dataset['FamilySize'] == 1, 'IsAlone'] = 1

        # fill Nan with mode
        self.dataset['Embarked'] = self.dataset['Embarked'].fillna(self.dataset['Embarked'].mode()[0])

        # fill Nan with median
        self.dataset['Fare'] = self.dataset['Fare'].fillna(self.dataset['Fare'].median())
        # binning with qcut
        self.dataset['Fare'] = pd.qcut(self.dataset['Fare'], 4)

        # fill Nan with values from random distribution
        age_avg = self.dataset['Age'].mean()
        age_std = self.dataset['Age'].std()
        age_null_count = self.dataset['Age'].isnull().sum()
        rng = np.random.RandomState(42)
        age_null_random_list = rng.uniform(age_avg - age_std, age_avg + age_std, size=age_null_count)
        self.dataset['Age'][np.isnan(self.dataset['Age'])] = age_null_random_list

        # binning with cut
        self.dataset['Age'] = pd.cut(self.dataset['Age'], 5)

        # apply regex
        self.dataset['Title'] = self.dataset['Name'].apply(self.get_title)
        # replace
        self.dataset['Title'] = self.dataset['Title'].replace(['Lady', 'Countess', 'Capt', 'Col', 'Don',
                                                               'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'],
                                                               'Rare')
        # replace
        self.dataset['Title'] = self.dataset['Title'].replace(['Mlle', 'Ms'], 'Miss')
        # replace
        self.dataset['Title'] = self.dataset['Title'].replace('Mme', 'Mrs')
        # fill nans
        self.dataset['Title'] = self.dataset['Title'].fillna(0)

        # drop columns
        drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp',
                         'Parch', 'FamilySize']

        self.dataset = self.dataset.drop(drop_elements, axis=1)

        # encode labels
        le = LabelEncoder()

        le.fit(self.dataset['Sex'])
        self.dataset['Sex'] = le.transform(self.dataset['Sex'])

        le.fit(self.dataset['Title'])
        self.dataset['Title'] = le.transform(self.dataset['Title'])

        le.fit(self.dataset['Embarked'].values)
        self.dataset['Embarked'] = le.transform(self.dataset['Embarked'].values)

        le.fit(self.dataset['Fare'])
        self.dataset['Fare'] = le.transform(self.dataset['Fare'])

        le.fit(self.dataset['Age'])
        self.dataset['Age'] = le.transform(self.dataset['Age'])

        return self.dataset

data_loader = DataLoader()
data_loader.fit(train)
result_dataset = data_loader.load_data()
result_dataset.head(10)

<ipython-input-33-d78411095ce0>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.dataset['Age'][np.isnan(self.dataset['Age'])] = age_null_random_list


,Survived,Pclass,Sex,Age,Fare,Embarked,IsAlone,Title
0,0,3,1,1,0,2,0,2
1,1,1,0,2,3,0,0,3
2,1,3,0,1,1,2,1,1
3,1,1,0,2,3,2,0,3
4,0,3,1,2,1,2,1,2
5,0,3,1,1,1,1,1,2
6,0,1,1,3,3,2,1,2
7,0,3,1,0,2,2,0,0
8,1,3,0,1,1,2,0,3
9,1,2,0,0,2,0,0,3
